In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np

c:\Users\javier\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ======== 1. Load IndoBERT dan model hasil training kamu ========
MODEL_NAME = "indobenchmark/indobert-base-p1"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model.load_weights("result/tf_model.h5")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# ======== 2. Fungsi prediksi sentimen ========
label_map = {0: "Negatif", 1: "Netral", 2: "Positif"}

def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="tf",
        padding=True,
        truncation=True,
        max_length=128
    )
    outputs = model(inputs)
    probs = tf.nn.softmax(outputs.logits, axis=-1)
    label_id = tf.argmax(probs, axis=1).numpy()[0]
    confidence = float(tf.reduce_max(probs).numpy())
    return label_map[label_id], confidence

In [9]:
ASPECT_KEYWORDS = {
    "aroma": ["aroma", "wangi", "bau", "harum", "semerbak", "wanginya"],
    "harga": ["harga", "mahal", "murah", "diskon", "promo"],
    "kemasan": ["kemasan", "botol", "desain", "tutup", "packaging", "kotak"],
    "ketahanan": ["tahan", "daya tahan", "lama", "cepat hilang", "awet"],
    "kualitas": ["kualitas", "bagus", "jelek", "baik", "buruk", "premium"]
}


In [5]:
# ======== 4. Fungsi deteksi aspek dari teks ========
def detect_aspect(text):
    found_aspects = []
    lower_text = text.lower()
    for aspect, keywords in ASPECT_KEYWORDS.items():
        if any(word in lower_text for word in keywords):
            found_aspects.append(aspect)
    return found_aspects if found_aspects else ["umum"]

In [10]:
# ======== 5. Fungsi ABSA utama ========
def absa_predict(text):
    aspects = detect_aspect(text)
    results = {}

    for aspect in aspects:
        # Ambil subteks terkait aspek
        sentences = [s for s in text.split('.') if aspect in s.lower()]
        sub_text = " ".join(sentences) if sentences else text

        sentiment, conf = predict_sentiment(sub_text)
        results[aspect] = {"sentiment": sentiment, "confidence": round(conf, 3)}
    return results

In [11]:
# ======== 6. Testing dengan input user ========
if __name__ == "__main__":
    print("=== Aspect-Based Sentiment Analysis (ABSA) ===")
    user_text = input("Masukkan review parfum: ")

    results = absa_predict(user_text)
    print("\nHasil Analisis:")
    for aspect, info in results.items():
        print(f"- {aspect.capitalize()} → {info['sentiment']} (confidence: {info['confidence']})")

=== Aspect-Based Sentiment Analysis (ABSA) ===

Hasil Analisis:
- Aroma → Positif (confidence: 0.951)
- Kualitas → Positif (confidence: 0.951)
